In [7]:
import os
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate   # type: ignore
from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser, PydanticOutputParser
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from typing import TypedDict
from langchain_groq import ChatGroq

from langchain.output_parsers import PydanticOutputParser

from pydantic import BaseModel, Field


load_dotenv()

True

In [8]:
llama ='llama3-70b-8192'


model = ChatGroq(temperature=0, model_name=llama)

## String Parser

In [32]:
prompt = ChatPromptTemplate.from_template('''Extract key information from the below text and give output in key value format.
                                          
                                        
                                  
                                              
                                               {topic}''')

output_parser = StrOutputParser()

chain = prompt | model | output_parser

response = chain.invoke({"topic": '''The mutual fund generated an 8.4% return in Q1 2025, with Chinese stocks increasing by 12%. The expense ratio is 0.95%, and 45% of the portfolio is in technology stocks. 
        The fund has a turnover rate of 25%, and its average yield is 4.2%. The price-to-earnings (P/E) ratio is 15, with a dividend yield of 2.1%. 
        The fund also has a beta of 1.1.'''})

print(response)
print(type(response))

Here is the extracted key information in key-value format:

**Return in Q1 2025**: 8.4%
**Chinese Stocks Return**: 12%
**Expense Ratio**: 0.95%
**Technology Stocks Allocation**: 45%
**Turnover Rate**: 25%
**Average Yield**: 4.2%
**Price-to-Earnings (P/E) Ratio**: 15
**Dividend Yield**: 2.1%
**Beta**: 1.1
<class 'str'>


## Json Output Parser

In [ ]:
prompt = ChatPromptTemplate.from_template('''Extract key information from the below text and give output in key value format.   
                                           Please give me the output in proper JSON format without any other explanations or headings.  
                                                                        
                                    
                                              
                                               {topic}''')

output_parser = JsonOutputParser()

chain = prompt | model | output_parser

response = chain.invoke({"topic": '''The mutual fund generated an 8.4% return in Q1 2025, with Chinese stocks increasing by 12%. The expense ratio is 0.95%, and 45% of the portfolio is in technology stocks. 
        The fund has a turnover rate of 25%, and its average yield is 4.2%. The price-to-earnings (P/E) ratio is 15, with a dividend yield of 2.1%. 
        The fund also has a beta of 1.1.'''})

print(response)
print(type(response))

OutputParserException: Invalid json output: Here is the extracted key information in key-value format:

**Return in Q1 2025**: 8.4%
**Chinese Stocks Return**: 12%
**Expense Ratio**: 0.95%
**Technology Stocks Allocation**: 45%
**Turnover Rate**: 25%
**Average Yield**: 4.2%
**Price-to-Earnings (P/E) Ratio**: 15
**Dividend Yield**: 2.1%
**Beta**: 1.1
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

In [11]:
response['Chinese Stocks Return']

'12%'

## Structured Output Parser

In [37]:
schema = [
    ResponseSchema(name='return_q1_2025', description='Return in Q1 2025'),
    ResponseSchema(name='chinese_stock_growth', description='Growth of Chinese stocks'),
]


# Step 2: Create parser
parser = StructuredOutputParser.from_response_schemas(schema)

format_instructions = parser.get_format_instructions()

In [39]:
prompt = ChatPromptTemplate.from_template('''Extract key information from the below topic.                                  
                                              Topic:
                                               {topic}
                                          
                                          {format_instructions}''')

chain = prompt | model | parser


response = chain.invoke({"topic": '''The mutual fund generated an 8.4% return in Q1 2025, with chinese stocks increasing by 12%. The expense ratio is 0.95%, and 45% of the portfolio is in technology stocks. 
        The fund has a turnover rate of 25%, and its average yield is 4.2%. The price-to-earnings (P/E) ratio is 15, with a dividend yield of 2.1%. 
        The fund also has a beta of 1.1.''', 'format_instructions': format_instructions})

print(response)

{'return_q1_2025': '8.4%', 'chinese_stock_growth': '12%', 'expense_ratio': '0.95%', 'technology_stock_allocation': '45%', 'turnover_rate': '25%', 'average_yield': '4.2%', 'price_to_earnings_ratio': '15', 'dividend_yield': '2.1%', 'beta': '1.1'}


In [14]:
type(response)

dict

## Pydantic Parser

In [ ]:
class FundInfo(BaseModel):
    return_q1_2025: str = Field(description="Return in Q1 2025")
    chinese_stock_growth: str = Field(description="Growth of Chinese stocks")
    expense_ratio: str = Field(description="Expense ratio of the fund")
    

parser = PydanticOutputParser(pydantic_object=FundInfo)


format_instructions_pyd = parser.get_format_instructions()

In [43]:
prompt = ChatPromptTemplate.from_template('''Extract key information from the below topic.                                  
                                              Topic:
                                               {topic}
                                          
                                          {format_instructions}''')

chain = prompt | model | parser


response = chain.invoke({"topic": '''The mutual fund generated an 8.4% return in Q1 2025, with US stocks increasing by 12%. The expense ratio is 0.95%, and 45% of the portfolio is in technology stocks. 
        The fund has a turnover rate of 25%, and its average yield is 4.2%. The price-to-earnings (P/E) ratio is 15, with a dividend yield of 2.1%. 
        The fund also has a beta of 1.1.''', 'format_instructions': format_instructions_pyd})



In [44]:
print(response) 

return_q1_2025='8.4%' chinese_stock_growth='N/A' expense_ratio='0.95%'


In [22]:
response.return_q1_2025

'8.4%'